# The RAG Flow Cleaning and Modularizing Code


In [20]:
import json
import os
import minsearch
from openai import OpenAI
from dotenv import load_dotenv


In [21]:
def read_and_parse_json(filename):
    with open(filename, 'rt') as f_in:
        docs_raw = json.load(f_in)
    
    documents = []

    for course_dict in docs_raw:
        for doc in course_dict['documents']:
            doc['course'] = course_dict['course']
            documents.append(doc)

    return documents

In [22]:
def index_and_fit(documents):
    index = minsearch.Index(
        text_fields=["question", "text", "section"],
        keyword_fields=["course"]
    )

    index.fit(documents)

    return index

In [23]:
def create_openai_client():
    load_dotenv()
    api_key = os.getenv('OPENAI_API_KEY')
    return OpenAI(api_key=api_key)

In [24]:
# Search function that takes a query and returns the search results
def search(query, index):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [25]:
# Function to build the prompt. It takes the query and the search results and returns the prompt
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. A student asks you the following question.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT: 
    {context}
    """.strip()

    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [26]:
# Function to generate the answer using the OpenAI API
def llm(prompt, client):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": prompt},
        ]
    )

    return response.choices[0].message.content

In [27]:
# This function takes a query and returns the answer using the LLM model
# It uses the search, build_prompt, and llm functions
def rag(query, index, client): 
    search_results = search(query, index)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt, client)
    return answer

In [28]:
documents = read_and_parse_json('documents.json')
index = index_and_fit(documents)
client = create_openai_client()
query = "how do I run kafka?"

print(rag(query, index, client))

To run Kafka, the exact method will depend on whether you are using Java or Python. Here are the steps for both:

### Java Kafka
1. Navigate to your project directory.
2. Use the following command in the terminal to run the Java producer:
   ```bash
   java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java
   ```

Replace `<jar_name>` with the actual name of your JAR file.

### Python Kafka
1. First, create a virtual environment and install the required packages:
    ```bash
    python -m venv env
    source env/bin/activate  # For MacOS/Linux
    # or
    env\Scripts\activate  # For Windows
    pip install -r path/to/requirements.txt
    ```

2. Make sure that Docker images required for your Kafka setup are up and running.
3. After the virtual environment is activated, you can run your Python Kafka application.

4. If you encounter a "Permission denied" error with `./build.sh`, you can fix it by running:
    ```bash
    chmod +x build.sh
    ``